In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import re
import fnmatch
import errno    
import json
import glob
from riboraptor.sradb import SRAdb
from riboraptor.helpers import path_leaf
from collections import defaultdict
from riboraptor.utils import summary_starlogs_over_runs

Populating the interactive namespace from numpy and matplotlib


In [ ]:
summary_starlogs_over_runs('/staging/as/skchoudh/re-ribo-analysis/hg38/SRP017942/starlogs/')

In [4]:
datasets = {}
datasets['hg38'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='hg38')
datasets['mm10'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='mm10')
datasets['mm10'] = pd.read_excel('../data/datasets/re-ribo-datasets.xlsx', sheet_name='mm10')


In [5]:
geodb = SRAdb('/staging/as/skchoudh/GEOmetadb.sqlite')
sradb = SRAdb('/staging/as/skchoudh/SRAmetadb.sqlite')

re_ribo_analysis_dir = '/staging/as/skchoudh/re-ribo-analysis/'

In [6]:
geodb.get_query("SELECT * FROM gse WHERE gse='GSE22004'")

,ID,contact,contributor,gse,last_update_date,overall_design,pubmed_id,repeats,repeats_sample_list,status,submission_date,summary,supplementary_file,title,type,variable,variable_description,web_link
0,19587.0,Name: Huili Guo;\tEmail: hguo@mit.edu;\tPhone:...,None,GSE22004,2013-06-07,Refer to individual Series.,20703300,None,None,Public on Aug 03 2010,2010-05-26,MicroRNAs (miRNAs) are endogenous ~22-nucleoti...,ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE22nnn...,Mammalian microRNAs predominantly act to decre...,Expression profiling by high throughput sequen...,None,None,None


In [7]:
df = pd.read_html('../data/datasets/hrpdviewer.html')

In [8]:
df = df[0].sort_values(by='Year')


In [12]:
df['SRP'] = df['GEO number'].apply(lambda x: geodb.convert_gse_to_srp(x))

In [13]:
df[df['GEO number'].str.contains('SRP')]['SRP'] = df[df['GEO number'].str.contains('SRP')]['GEO number']

/home/cmb-panasas2/skchoudh/software_frozen/anaconda27/envs/riboraptor/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
df.to_csv('../data/datasets/hrpdviewer.tsv', sep='\t', header=True, index=False)

In [20]:
[len(df['GEO number'].unique())]

68

In [24]:
datasets = defaultdict(list)

In [26]:
for directory in glob.glob('{}/*/*'.format(re_ribo_analysis_dir)):
    srp = path_leaf(directory)
    assembly = path_leaf(os.path.dirname(directory))
    datasets[srp].append(assembly)

In [27]:
datasets

defaultdict(list,
            {'SRP003554': ['mm10'],
             'SRP007567': ['mm10'],
             'SRP010679_fixed_adapters': ['hg38'],
             'SRP017942': ['hg38', 'mm10'],
             'SRP021058': ['mm10'],
             'SRP028243': ['BDGP6'],
             'SRP028243_fixed_adapters': ['BDGP6'],
             'SRP030014': ['mm10'],
             'SRP031501': ['hg38', 'mm10'],
             'SRP033366': ['BDGP6'],
             'SRP038695': ['hg38'],
             'SRP042937': ['hg38'],
             'SRP043036': ['sacCerR64'],
             'SRP045214': ['hg38'],
             'SRP045475': ['BDGP6'],
             'SRP048708': ['mm10'],
             'SRP049168': ['hg38', 'mm10'],
             'SRP055009': ['hg38'],
             'SRP055707': ['sacCerR64'],
             'SRP056576': ['mm10'],
             'SRP056647': ['sacCerR64'],
             'SRP062409': ['mm10'],
             'SRP063562': ['MG1655'],
             'SRP064304': ['mm10'],
             'SRP074765': ['mm10'],
       

In [2]:
file_name = '../data/datasets/rpfdb_metadata.json'
rpfdb_data = json.load(open(file_name))
rpfdb_df = pd.DataFrame(rpfdb_data)
rpfdb_df = rpfdb_df.drop(columns=['Abstract'])
rpfdb_df[['Author', 'Year']] = rpfdb_df.Author_year.str.split(',', n=1, expand=True)
rpfdb_df = rpfdb_df.drop(columns=['Author_year'])
rpfdb_df.Year = rpfdb_df.Year.astype(int)
species_value_counts = rpfdb_df.Species.value_counts() 
rpfdb_df.head()
rpfdb_df = rpfdb_df.sort_values(by=['Year', 'Species'])

rpfdb_df.Strain = [x.lower() for x in rpfdb_df.Strain]


In [5]:
rpfdb_df[rpfdb_df.Species == 'Human'].to_excel('../data/datasets/rpfdb_human.xls', index=False)

In [6]:
rpfdb_df[rpfdb_df.Species == 'Mouse'].to_excel('../data/datasets/rpfdb_mouse.xls', index=False)

In [7]:
rpfdb_df[rpfdb_df.Species == 'Drosophila'].to_excel('../data/datasets/rpfdb_drosophila.xls', index=False)

In [8]:
rpfdb_df[rpfdb_df.Species == 'Zebrafish'].to_excel('../data/datasets/rpfdb_zebrafish.xls', index=False)

In [9]:
rpfdb_df[rpfdb_df.Species == 'Rat'].to_excel('../data/datasets/rpfdb_rat.xls', index=False)

In [10]:
rpfdb_df[rpfdb_df.Species == 'Chicken'].to_excel('../data/datasets/rpfdb_chicken.xls', index=False)

In [11]:
rpfdb_df[rpfdb_df.Species == 'C.albicans'].to_excel('../data/datasets/rpfdb_c_albicans.xls', index=False)

In [12]:
rpfdb_df[rpfdb_df.Species == 'C.elegans'].to_excel('../data/datasets/rpfdb_c_elegans.xls', index=False)

In [14]:
rpfdb_df.Species.unique()

array(['S.cerevisiae', 'Human', 'Mouse', 'E.coli', 'B.subtilis',
       'C.elegans', 'Zebrafish', 'Arabidopsis', 'Drosophila', 'T.brucei',
       'C.albicans', 'C.crescentus', 'M.smegmatis', 'P.falciparum ',
       'S.aureus', 'S.pombe', 'X.laevis', 'C.reinhardtii', 'N.crassa',
       'Rat', 'Hamster', 'M.abscessus', 'S.coelicolor', 'Chicken',
       'H.salinarum', 'P.aeruginosa', 'P.fluorescens', 'S.enterica',
       'T.gondii'], dtype=object)

In [6]:
rpfdb_df[rpfdb_df.Species == 'Rat']

,Citation,Index,Ref,Samples,Species,Strain,Study,Title,Treatment,Year,Author
224,Integrated Transcriptome and Proteome Analyses...,213,Rnor6.0,12,Rat,"brain, liver tissue",GSE66715,Integrated Transcriptome and Proteome Analyses...,cycloheximide,2015,Ori A
225,Oxygen and glucose deprivation induces widespr...,212,Rnor6.0,8,Rat,PC12 cells,GSE60752,Time resolved ribosome profiling study of oxyg...,100 μg/ml streptomycin,2015,Andreev DE
226,Translational regulation shapes the molecular ...,214,Rnor6.0,20,Rat,strain: BN/SHR,PRJEB7498,Ribosome profiling in mammalian tissue reveals...,0.1 mg/ml cycloheximide,2015,Schafer S


In [13]:
rpfdb_df[rpfdb_df.Species == 'Hamster']

,Citation,Index,Ref,Samples,Species,Strain,Study,Title,Treatment,Year,Author
25,Ribosome profiling-guided depletion of an mRNA...,247,CriGri1.0,6,Hamster,strain: cs13-1.0,GSE79512,Ribosome profiling-guided depletion of an mRNA...,100 μg/mL cycloheximide,2016,Kallehauge TB


In [8]:
rpfdb_df[rpfdb_df.Species == 'Drosophila']

,Citation,Index,Ref,Samples,Species,Strain,Study,Title,Treatment,Year,Author
35,Ribosome profiling reveals pervasive and regul...,169,BDGP6,10,Drosophila,"embryo, S2 cell line",GSE49197,Ribosome profiling reveals pervasive and regul...,20 μg/ml emetine,2013,Dunn JG
37,Widespread changes in the posttranscriptional ...,170,BDGP6,6,Drosophila,"Mature oocytes, activated eggs",GSE52799,Polysome profiling and ribosome footprinting o...,100 μg/mL cycloheximide,2014,Kronja I
32,Modified ribosome profiling reveals high abund...,174,BDGP6,2,Drosophila,Kc167 cell,PRJEB5938,Ribosome profiling with MN and RNaseI,250-μM cycloheximide,2015,Miettinen TP
33,Modified ribosome profiling reveals high abund...,175,BDGP6,3,Drosophila,Kc167 cell,PRJEB7901,Analysis of conditions leading to 3'UTR riboso...,250-μM cycloheximide,2015,Miettinen TP
34,mRNA poly(A)-tail changes specified by deadeny...,172,BDGP6,14,Drosophila,oocytes/embryos/eggs,GSE83616,mRNA Poly(A)-tail Changes Specified by Deadeny...,100 μg/mL cycloheximide,2016,Eichhorn SW
31,Development of a tissue-specific ribosome prof...,173,BDGP6,9,Drosophila,Body wall muscle,GSE99920,Transcriptional and ribosome profiling of Dros...,100 μg/mL cycloheximide,2017,Chen X
36,Translation of CircRNAs. Mol Cell 2017 Apr 6;6...,171,BDGP6,4,Drosophila,Fly heads,GSE79626,Ribosome footprinting of fly heads,100 ug/ml cycloheximide,2017,Pamudurti NR


In [28]:
rpfdb_df.Strain.unique()

array(['strain: by4741', 'hela cells',
       'neutrophils cultured from mouse bone marrow', 'strain: mc4100',
       'embryonic stem cell', 'strain: 168', 'strain: n2',
       'strain: mg1655, bw25113', 'hek293 / hela cells',
       'hcmv-infected human foreskin fibroblasts', 'thp-1 cell',
       'hek293 cells', 'pc3 cells', 'embryonic fibroblast', 'mef cells',
       'strain: sk1', 'strain: tuab', 'whole seedlings',
       'etiolated seedling', 'strain: 3610-δepsh, 3610-δsera',
       'strain: n2, af16, pb4641, pb2801', 'embryo, s2 cell line',
       'strain: mg1655', 'peo1 / c4-4 celss', 'mm1.s myeloma cells',
       'bj cells', 'bj / cybrid cells', 'hek293t cells', '3t3 cells',
       'strain: by4716', 'strain: s288c',
       's. cerevisiae by4176 & s. paradoxus cbs432',
       'strain: s288c,cbs432,gsy88', 'strain: lister427', 'strain: tl/ab',
       'strain: sc5314', 'strain: na1000',
       'mature oocytes, activated eggs', 'strain: mg1655, ep61',
       'strain: bw25113', 'sw13

In [25]:
rpfdb_df[rpfdb_df.Strain.str.contains('brain')]

,Citation,Index,Ref,Samples,Species,Strain,Study,Title,Treatment,Year,Author
120,Ribosome profiling reveals a cell-type-specifi...,24,GRCh38,5,Human,brain tissue,GSE51424,Cell type-specific ribosome profiling in vivo.,0.1 mg/ml cycloheximide,2014,Gonzalez C
203,Ribosome profiling reveals a cell-type-specifi...,115,GRCm38,5,Mouse,brain tissue,GSE51424,Cell type-specific ribosome profiling in vivo.,0.1 mg/ml cycloheximide,2014,Gonzalez C
168,A Dynamic Unfolded Protein Response Contribute...,138,GRCm38,4,Mouse,e14.5 mouse embryonic forebrains,GSE74683,A dynamic unfolded protein response contribute...,100 µg/mL cycloheximide,2015,Laguesse S
224,Integrated Transcriptome and Proteome Analyses...,213,Rnor6.0,12,Rat,"brain, liver tissue",GSE66715,Integrated Transcriptome and Proteome Analyses...,cycloheximide,2015,Ori A
187,Ligation-free ribosome profiling of cell type-...,139,GRCm38,12,Mouse,mouse brain,GSE78163,Ligation-Free Ribosome Profiling Reveals Cell ...,0.1 mg/ml cycloheximide,2016,Hornstein N


In [32]:
rpfdb_df[rpfdb_df.Strain.str.contains('fibroblasts')].sort_values(by='Strain')

,Citation,Index,Ref,Samples,Species,Strain,Study,Title,Treatment,Year,Author
210,The unfolded protein response triggers selecti...,117,GRCm38,30,Mouse,embryonic fibroblasts,GSE53743,Translational profiling in the unfolded protei...,180μM cycloheximide,2014,Reid DW
175,Complementary Roles of GADD34- and CReP-Contai...,130,GRCm38,20,Mouse,embryonic fibroblasts,GSE69800,Ribosome profiling analysis of GADD34 null cells,50 μg/ml cycloheximide,2015,Reid DW
165,Unavailable,128,GRCm38,60,Mouse,embryonic fibroblasts,GSE68265,Dissection of the translational impacts of the...,50 μg/ml cycloheximide,2016,Reid DW
77,Decoding human cytomegalovirus. Science 2012 N...,7,GRCh38,10,Human,hcmv-infected human foreskin fibroblasts,GSE41605,Decoding human cytomegalovirus using ribosome ...,cycloheximide/Harringtonine/lactimidomycin,2012,Stern-Ginossar N
166,Unavailable,140,GRCm38,3,Mouse,mouse embryonc fibroblasts,GSE80156,A coding sequence-embedded principle governs t...,100 ug/ml cycloheximide,2016,Wan J
140,The Transcription and Translation Landscapes d...,46,GRCh38,7,Human,primary fibroblasts,GSE69906,The transcription and translation landscapes d...,100μg/ml cycloheximide,2015,Tirosh O
160,Widespread disruption of host transcription te...,38,GRCh38,12,Human,primary human foreskin fibroblasts (hff),GSE60040,Wide-spread disruption of transcription termin...,50 μg/ml cycloheximide,2015,Rutkowski AJ


In [26]:
rpfdb_df[rpfdb_df.Strain.str.contains('kidney')]


,Citation,Index,Ref,Samples,Species,Strain,Study,Title,Treatment,Year,Author
153,Tumour-specific proline vulnerability uncovere...,57,GRCh38,16,Human,kidney tissue,GSE59820,Ribosome profiling on sections taken from a ki...,"100μg/ml chloramphenicol, 100μg/ml cycloheximide",2016,Loayza-Puch F
137,The novel lysine specific methyltransferase ME...,82,GRCh38,4,Human,kidney tissue,GSE93133,The novel lysine specific methyltransferase ME...,100 g/ml cycloheximide,2017,Malecki J
215,Translational contributions to tissue specific...,141,GRCm38,24,Mouse,kidney tissue,GSE81283,Translational contributions to tissue-specific...,100 µg/mL cycloheximide,2017,Castelo-Szekely V


In [27]:
rpfdb_df[rpfdb_df.Strain.str.contains('liver')]

,Citation,Index,Ref,Samples,Species,Strain,Study,Title,Treatment,Year,Author
192,mRNA destabilization is the dominant effect of...,122,GRCm38,5,Mouse,liver tissue / 3t3 cells / b cells,GSE60426,mRNA destabilization is the dominant effect of...,cycloheximide,2014,Eichhorn SW
198,Poly(A)-tail profiling reveals an embryonic sw...,116,GRCm38,1,Mouse,3t3 cells / liver tissue,GSE52809,Poly(A)-tail profiling reveals an embryonic sw...,100 µg/ml cycloheximide,2014,Subtelny AO
174,Circadian and feeding rhythms differentially a...,135,GRCm38,84,Mouse,liver tissue,GSE73553,Circadian and feeding rhythms differentially a...,20 µg/mL cycloheximide,2015,Atger F
176,Dietary Selenium Levels Affect Selenoprotein E...,131,GRCm38,8,Mouse,liver tissue,GSE70159,Dietary selenium levels affect selenoprotein e...,200 µg/ml cycloheximide,2015,Tsuji PA
205,Ribosome profiling reveals the rhythmic liver ...,126,GRCm38,24,Mouse,liver tissue,GSE67305,Ribosome profiling reveals the rhythmic liver ...,100 μg/ml cycloheximide,2015,Janich P
224,Integrated Transcriptome and Proteome Analyses...,213,Rnor6.0,12,Rat,"brain, liver tissue",GSE66715,Integrated Transcriptome and Proteome Analyses...,cycloheximide,2015,Ori A
201,Ribonuclease selection for ribosome profiling....,142,GRCm38,3,Mouse,liver tissue,GSE82220,Ribonuclease Selection for Ribosome Profiling,0.1 mg/ml cycloheximide,2016,Gerashchenko MV
209,The RNA-binding protein Secisbp2 differentiall...,146,GRCm38,9,Mouse,liver tissue,GSE84112,Global analysis of selenoprotein expression re...,200 µg/ml cycloheximide,2016,Fradejas-Villar N
194,Multiple RNA structures affect translation ini...,106,GRCm38,12,Mouse,liver tissue from mice carrying genomic deleti...,GSE102890,Multiple RNA structures affect translation ini...,200 ug/ml cycloheximide,2017,Mariotti M
196,N6-Methyladenosine Guides mRNA Alternative Tra...,105,GRCm38,10,Mouse,liver tissue / mouse embryonic fibroblast,GSE102659,N6-Methyladenosine Guides mRNA Alternative Tra...,100 µg/ml cycloheximide,2017,Zhou J


In [4]:
rpfdb_df[rpfdb_df.Species == 'Mouse']

,Citation,Index,Ref,Samples,Species,Strain,Study,Title,Treatment,Year,Author
190,Mammalian microRNAs predominantly act to decre...,107,GRCm38,2,Mouse,Neutrophils cultured from mouse bone marrow,GSE22001,Analysis of mir-223 knockout cultured neutroph...,100 ug/ml cycloheximide,2010,Guo H
202,Ribosome profiling of mouse embryonic stem cel...,108,GRCm38,13,Mouse,Embryonic stem cell,GSE30839,Ribosome Profiling of Mouse Embryonic Stem Cel...,"2 μg/ml harringtonine, 100 μg/ml cycloheximide...",2011,Ingolia NT
170,A unifying model for mTORC1-mediated regulatio...,110,GRCm38,8,Mouse,Embryonic fibroblast,GSE36892,A unifying model for mTORC1-mediated regulatio...,10 ug/ml cycloheximide,2012,Thoreen CC
183,Global mapping of translation initiation sites...,152,GRCm38,5,Mouse,MEF cells,SRA056377,Global mapping of translation initiation sites...,"cycloheximide, lactimidomycin",2012,Lee S
188,LIN28A is a suppressor of ER-associated transl...,111,GRCm38,2,Mouse,Embryonic stem cell,GSE37111,Transcriptome-wide identification of LIN28A ta...,100 μg/ml cycloheximide,2012,Cho J
181,Global analyses of UPF1 binding and function r...,113,GRCm38,3,Mouse,Embryonic stem cell,GSE41785,Global analysis of Upf1 in mESCs reveals expan...,100ug/ml cycloheximide,2013,Hurt JA
216,Widespread regulation of translation by elonga...,109,GRCm38,4,Mouse,3T3 cells,GSE32060,Widespread regulation of translation by elonga...,"0.1 mM harringtonine, 0.1 mg/ml cycloheximide,...",2013,Shalgi R
172,Assessing the translational landscape of myoge...,155,GRCm38,12,Mouse,murine C2C12 cell,PRJEB7207,Ribosome profiling in C2C12 mouse proliferatin...,"cycloheximide, harringtonin",2014,de Klerk E
192,mRNA destabilization is the dominant effect of...,122,GRCm38,5,Mouse,liver tissue / 3T3 cells / B cells,GSE60426,mRNA destabilization is the dominant effect of...,cycloheximide,2014,Eichhorn SW
195,Musashi proteins are post-transcriptional regu...,119,GRCm38,8,Mouse,Neural stem cells,GSE58423,Musashi proteins are post-transcriptional regu...,cycloheximide,2014,Katz Y


# Convert GSE to SRP